In [2]:
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np

## Define techinical indicators

In [3]:
def MACD(DF, a, b, c):
    intra_price = DF.copy()
    intra_price["MA_FAST"] = intra_price["Adj Close"].ewm(span = a, min_periods = a).mean()
    intra_price["MA_SLOW"] = intra_price["Adj Close"].ewm(span = b, min_periods = b).mean()
    intra_price["MACD"] = intra_price["MA_FAST"] - intra_price["MA_SLOW"]
    intra_price["SIGNAL"] = intra_price["Adj Close"].ewm(span = c, min_periods = c).mean()
    intra_price.dropna(inplace=True)
    return intra_price

def SMA(DF, day):
    copy = DF.copy()
    sma = copy.rolling(window=day, min_periods=day).mean() 
    return sma

def EMA(DF, day):
    copy = DF.copy()
    sma = copy.ewm(span=day, min_periods=day).mean() 
    return sma

def ATR(DF, n):
    df = DF.copy()
    df["H-L"] = abs(df["High"] - df["Low"]).shift(1)
    df["H-PC"] = abs(df["High"] - df["Adj Close"]).shift(1)
    df["L-PC"] = abs(df["Low"] - df["Adj Close"]).shift(1)
    df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].rolling(n).mean()
    df2 = df.drop(["H-L", "H-PC", "L-PC"], axis=1)
    return df2

def BollBnd(DF, n):
    df = DF.copy()
    df["MA"] = df["Adj Close"].rolling(n).mean()
    df["BB_UP"] = df["MA"] + 2*df["MA"].rolling(n).std()
    df["BB_DOWN"] = df["MA"] - 2*df["MA"].rolling(n).std()
    df["BB_RANGE"] = df["BB_UP"] - df["BB_DOWN"]
    df.dropna(inplace=True)
    return df

def RSI(DF, n):
    df = DF.copy()
    print(len(df))
    df["delta"] = df["Adj Close"] - df["Adj Close"].shift(1)
    df["gain"] = np.where(df["delta"]>=0, df["delta"], 0)
    df["loss"] = np.where(df["delta"]<0, abs(df["delta"]), 0)
    avg_gain = []
    avg_loss = []
    gain = df["gain"].tolist()
    loss = df["loss"].tolist()
    for i in range(len(df)):
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df["gain"].rolling(n).mean().tolist()[n])
            avg_loss.append(df["loss"].rolling(n).mean().tolist()[n])
        elif i > n:
            avg_gain.append((avg_gain[i - 1]*(n - 1) + gain[i])/n)
            avg_loss.append((avg_loss[i - 1]*(n - 1) + loss[i])/n)
    df["avg_gain"] = np.array(avg_gain)

    df["avg_loss"] = np.array(avg_loss)
    df["RS"] = df["avg_gain"]/df["avg_loss"]
    df["RSI"] = 100 - (100/(1+df["RS"]))
    return df["RSI"]

def ADX(DF, n):
    df2 = DF.copy()
    df2["TR"] = ATR(df2, n)["TR"]
    df2["DMplus"] = np.where((df2["High"] - df2["High"].shift(1))>(df2["Low"].shift(1)-df2["Low"]), 
                             df2["High"] - df2["High"].shift(1),
                             0)
    df2["DMplus"] = np.where(df2["DMplus"]<0, 0, df2["DMplus"])
    df2["DMminus"] = np.where((df2["Low"].shift(1)-df2["Low"])>(df2["High"] - df2["High"].shift(1)),
                             df2["Low"].shift(1)-df2["Low"],
                             0)
    df2["DMminus"] = np.where(df2["DMminus"]<0, 0, df2["DMminus"])
    TRn=[]
    DMplusN=[]
    DMminusN=[]
    TR = df2["TR"].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i<n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2["TR"].rolling(n).sum().tolist()[n])
            DMplusN.append(df2["DMplus"].rolling(n).sum().tolist()[n])
            DMminusN.append(df2["DMminus"].rolling(n).sum().tolist()[n])
        else:
            TRn.append(TRn[i-1] - (TRn[i-1]/14) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/14) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/14) + DMminus[i])
            
    df2["TRn"] = np.array(TRn)
    df2["DMplusN"] = np.array(DMplusN)
    df2["DMminusN"] = np.array(DMminusN)
    df2["DIplusN"] = (100*df2["DMplusN"]/df2["TRn"])
    df2["DIminusN"] = (100*df2["DMminusN"]/df2["TRn"])
    df2["DIsum"] =  df2["DIplusN"] + df2["DIminusN"] 
    df2["DIdiff"] =  abs(df2["DIplusN"] - df2["DIminusN"])
    df2["DX"] = 100*(df2["DIdiff"]/df2["DIsum"])
    ADX = []
    DX = df2["DX"].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2["DX"][j-n+1: j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n - 1)*ADX[j-1] + DX[j])/n)
    df2["ADX"] =np.array(ADX)
    return df2["ADX"]
        

## Try on one single stock microsoft for one year

In [4]:
start = dt.datetime.today() - dt.timedelta(5000)
end = dt.datetime.today()
MSTF_cl = pd.DataFrame()
MSTF_cl = yf.download("KNDI", start, end)
daily_return = MSTF_cl.pct_change()["Adj Close"]
MSTF_cl["RETURN"] = daily_return

[*********************100%***********************]  1 of 1 completed


In [5]:
MSTF_cl["MACD"] = MACD(MSTF_cl, 12, 26, 9)["SIGNAL"]
MSTF_cl["SMA"] = SMA(daily_return, 5)
MSTF_cl["EMA"] = EMA(daily_return, 5)
MSTF_cl = ATR(MSTF_cl, 20)
MSTF_cl = BollBnd(MSTF_cl,10)
MSTF_cl["RSI"] = np.array(RSI(MSTF_cl, 14))
MSTF_cl["ADX"] = np.array(ADX(MSTF_cl, 14))

3399


In [6]:
import seaborn as sns
import numpy as np # linear algebra
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
from sklearn import metrics
from sklearn.metrics import precision_score

In [7]:
df = MSTF_cl.dropna(axis=0)
df

,Open,High,Low,Close,Adj Close,Volume,RETURN,MACD,SMA,EMA,TR,ATR,MA,BB_UP,BB_DOWN,BB_RANGE,RSI,ADX
Date,,,,,,,,,,,,,,,,,,
2007-11-15,4.25,4.25,4.25,4.25,4.25,3500,0.024096,4.387728,0.007544,-0.001400,0.10,0.1910,4.450,4.869094,4.030906,0.838189,31.381650,31.807032
2007-11-16,4.25,4.25,4.17,4.17,4.17,1700,-0.018824,4.344182,-0.005977,-0.007208,0.00,0.1760,4.367,4.827566,3.906434,0.921132,29.647033,33.010565
2007-11-19,4.17,4.17,4.05,4.10,4.10,6900,-0.016787,4.295346,-0.007009,-0.010401,0.08,0.1750,4.277,4.762378,3.791622,0.970755,28.179284,34.434087
2007-11-20,4.00,4.55,4.00,4.40,4.40,24100,0.073171,4.316276,0.007626,0.017457,0.12,0.1760,4.237,4.716933,3.757067,0.959865,41.537667,33.336315
2007-11-21,4.05,4.05,3.72,4.05,4.05,38600,-0.079545,4.263021,-0.003578,-0.014877,0.55,0.1885,4.202,4.661403,3.742597,0.918807,33.669524,33.181130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-05,6.52,6.55,6.18,6.23,6.23,1443100,-0.040062,6.249286,0.011179,0.002211,0.26,0.4880,6.161,6.725232,5.596768,1.128465,43.494835,34.716302
2021-04-06,6.22,6.47,6.11,6.32,6.32,1452000,0.014446,6.263429,0.025198,0.006290,0.37,0.4530,6.113,6.664918,5.561082,1.103836,44.886394,34.576386
2021-04-07,6.30,6.46,6.18,6.22,6.22,1398500,-0.015823,6.254743,0.007034,-0.001081,0.36,0.4435,6.091,6.589397,5.592603,0.996794,43.601527,34.446464


In [8]:
y_t = np.array(df['RETURN'])

y_t = np.where(y_t < 0, 0, 1)

X_t = df
X_t = df.drop(["Open", "High", "Low", "Adj Close", "Volume", "RETURN" ],axis=1)
X_t = np.array(X_t)

lab_enc = preprocessing.LabelEncoder()
y_t = lab_enc.fit_transform(y_t)

print("shape of Y :"+ str(y_t.shape))
print("shape of X :"+ str(X_t.shape))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_t)

shape of Y :(3372,)
shape of X :(3372, 12)


In [17]:
# X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)
X_train = X_t[:2800]
X_test = X_t[2800:]
Y_train = y_t[:2800]
Y_test = y_t[2800:]

print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))


print(utils.multiclass.type_of_target(Y_train))
print(utils.multiclass.type_of_target(Y_test))


shape of X Train :(2800, 12)
shape of X Test :(572, 12)
shape of Y Train :(2800,)
shape of Y Test :(572,)
binary
binary


In [18]:
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='linear',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    Y_predict = clf.predict(X_test)
    precisiontest = precision_score(Y_predict, Y_test)
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_predict)
    roc = metrics.auc(fpr, tpr)
    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} ".format(this_C,scoretrain,scoretest))
    print("Test precision: {:2f}, Test roc: {:2f}\n".format(precisiontest, roc))

Linear SVM value of C:1, training score :0.705000 , Test Score: 0.716783 
Test precision: 0.625000, Test roc: 0.717430

Linear SVM value of C:3, training score :0.750714 , Test Score: 0.769231 
Test precision: 0.690972, Test roc: 0.769782

Linear SVM value of C:5, training score :0.770000 , Test Score: 0.790210 
Test precision: 0.725694, Test roc: 0.790664

Linear SVM value of C:10, training score :0.776071 , Test Score: 0.795455 
Test precision: 0.753472, Test roc: 0.795750

Linear SVM value of C:40, training score :0.791429 , Test Score: 0.811189 
Test precision: 0.791667, Test roc: 0.811326

Linear SVM value of C:60, training score :0.789286 , Test Score: 0.809441 
Test precision: 0.802083, Test roc: 0.809492

Linear SVM value of C:80, training score :0.791071 , Test Score: 0.804196 
Test precision: 0.795139, Test roc: 0.804260

Linear SVM value of C:100, training score :0.793214 , Test Score: 0.802448 
Test precision: 0.791667, Test roc: 0.802523



In [19]:
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='poly',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    Y_predict = clf.predict(X_test)
    precisiontest = precision_score(Y_predict, Y_test)
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_predict)
    roc = metrics.auc(fpr, tpr)
    print("Poly SVM value of C:{}, training score :{:2f} , Test Score: {:2f} ".format(this_C,scoretrain,scoretest))
    print("Test precision: {:2f}, Test roc: {:2f}\n".format(precisiontest, roc))

Poly SVM value of C:1, training score :0.762500 , Test Score: 0.784965 
Test precision: 0.697917, Test roc: 0.785578

Poly SVM value of C:3, training score :0.778929 , Test Score: 0.797203 
Test precision: 0.711806, Test roc: 0.797804

Poly SVM value of C:5, training score :0.782857 , Test Score: 0.802448 
Test precision: 0.725694, Test roc: 0.802988

Poly SVM value of C:10, training score :0.792143 , Test Score: 0.791958 
Test precision: 0.732639, Test roc: 0.792376

Poly SVM value of C:40, training score :0.797143 , Test Score: 0.797203 
Test precision: 0.770833, Test roc: 0.797388

Poly SVM value of C:60, training score :0.797500 , Test Score: 0.784965 
Test precision: 0.750000, Test roc: 0.785211

Poly SVM value of C:80, training score :0.802500 , Test Score: 0.790210 
Test precision: 0.756944, Test roc: 0.790444

Poly SVM value of C:100, training score :0.802857 , Test Score: 0.793706 
Test precision: 0.774306, Test roc: 0.793843



In [20]:
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='rbf',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    Y_predict = clf.predict(X_test)
    precisiontest = precision_score(Y_predict, Y_test)
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_predict)
    roc = metrics.auc(fpr, tpr)
    print("RBF SVM value of C:{}, training score :{:2f} , Test Score: {:2f}".format(this_C,scoretrain,scoretest))
    print("Test precision: {:2f}, Test roc: {:2f}\n".format(precisiontest, roc))

RBF SVM value of C:1, training score :0.738929 , Test Score: 0.737762
Test precision: 0.604167, Test roc: 0.738703

RBF SVM value of C:3, training score :0.772500 , Test Score: 0.784965
Test precision: 0.680556, Test roc: 0.785700

RBF SVM value of C:5, training score :0.785714 , Test Score: 0.793706
Test precision: 0.711806, Test roc: 0.794283

RBF SVM value of C:10, training score :0.800357 , Test Score: 0.788462
Test precision: 0.715278, Test roc: 0.788977

RBF SVM value of C:40, training score :0.821071 , Test Score: 0.811189
Test precision: 0.791667, Test roc: 0.811326

RBF SVM value of C:60, training score :0.825357 , Test Score: 0.819930
Test precision: 0.809028, Test roc: 0.820007

RBF SVM value of C:80, training score :0.832500 , Test Score: 0.823427
Test precision: 0.812500, Test roc: 0.823504

RBF SVM value of C:100, training score :0.838929 , Test Score: 0.819930
Test precision: 0.819444, Test roc: 0.819933



## Try on 6 stocks, 1000 days of data
- "MSFT", "TSLA", "^N225", "^DJI", "BTC-USD", "EURUSD=X"

In [12]:
start = dt.datetime.today() - dt.timedelta(1000)
end = dt.datetime.today()

In [13]:
def calTechnicalIndicator(stocks, start, end):
    cl = pd.DataFrame()
    sticker_map = {}
    for ticker in stocks:
        MSTF_cl = pd.DataFrame()
        MSTF_cl = yf.download(ticker, start, end)
        daily_return = MSTF_cl.pct_change()["Adj Close"]
        MSTF_cl["RETURN"] = daily_return
        MSTF_cl["MACD"] = MACD(MSTF_cl, 12, 26, 9)["SIGNAL"]
        MSTF_cl["SMA"] = SMA(daily_return, 5)
        MSTF_cl["EMA"] = EMA(daily_return, 5)
        MSTF_cl = ATR(MSTF_cl, 20)
        MSTF_cl = BollBnd(MSTF_cl,10)
        MSTF_cl["RSI"] = np.array(RSI(MSTF_cl, 14))
        MSTF_cl["ADX"] = np.array(ADX(MSTF_cl, 14))
        sticker_map.a
        cl = cl.append(MSTF_cl)
    return cl

In [14]:
stocks = ["MSFT", "TSLA", "^N225", "^DJI", "BTC-USD", "EURUSD=X" ]

In [15]:
df = calTechnicalIndicator(stocks, start, end)

[*********************100%***********************]  1 of 1 completed
664
[*********************100%***********************]  1 of 1 completed
664
[*********************100%***********************]  1 of 1 completed
639
[*********************100%***********************]  1 of 1 completed
664
[*********************100%***********************]  1 of 1 completed
972
[*********************100%***********************]  1 of 1 completed
669


In [16]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Adj Close,Volume,RETURN,MACD,SMA,EMA,TR,ATR,MA,BB_UP,BB_DOWN,BB_RANGE,RSI,ADX
Date,,,,,,,,,,,,,,,,,,
2018-08-16,108.300003,108.860001,107.300003,107.639999,104.268089,21384300,-0.000186,104.633042,-0.002925,-0.001876,4.702538,4.596897,104.901703,105.697325,104.106081,1.591244,57.992994,27.771456
2018-08-17,107.360001,107.900002,106.690002,107.580002,104.209976,18061500,-0.000557,104.548428,-0.001815,-0.001436,4.591911,4.598088,104.897265,105.744284,104.050246,1.694037,57.710991,26.250663
2018-08-20,107.510002,107.900002,106.480003,106.870003,103.522209,17914200,-0.006600,104.343183,-0.001686,-0.003157,3.690025,4.499908,104.815366,105.650653,103.980080,1.670573,54.342974,24.679578
2018-08-21,106.919998,107.349998,105.849998,105.980003,102.660103,22881900,-0.008328,104.006566,-0.005846,-0.004881,4.377792,4.521138,104.574884,105.367197,103.782571,1.584627,50.374440,23.083707
2018-08-22,105.849998,107.339996,105.779999,107.059998,103.706261,18000600,0.010191,103.946505,-0.001096,0.000143,4.689896,4.509016,104.380155,105.076521,103.683790,1.392731,54.697852,21.654217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-22,1.212445,1.215510,1.209219,1.212636,1.212636,0,0.002850,1.209875,0.000075,0.001020,0.006472,0.006203,1.210393,1.212770,1.208017,0.004754,52.073570,12.253703
2021-02-23,1.216693,1.217878,1.213681,1.216680,1.216680,0,0.003335,1.211236,0.000575,0.001792,0.006291,0.006173,1.211525,1.214715,1.208336,0.006380,56.159439,11.900740
2021-02-24,1.215200,1.217404,1.211130,1.215214,1.215214,0,-0.001204,1.212032,0.000989,0.000793,0.004198,0.006050,1.211847,1.215486,1.208208,0.007278,54.351492,11.069688


In [17]:
y_t = np.array(df['RETURN'])

y_t = np.where(y_t < 0, 0, 1)

X_t = df
X_t = df.drop(["Open", "High", "Low", "Adj Close", "Volume", "RETURN" ],axis=1)
X_t = np.array(X_t)

lab_enc = preprocessing.LabelEncoder()
y_t = lab_enc.fit_transform(y_t)

print("shape of Y :"+ str(y_t.shape))
print("shape of X :"+ str(X_t.shape))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_t)

shape of Y :(4110,)
shape of X :(4110, 12)


In [18]:
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))


print(utils.multiclass.type_of_target(Y_train))
print(utils.multiclass.type_of_target(Y_test))

shape of X Train :(3288, 12)
shape of X Test :(822, 12)
shape of Y Train :(3288,)
shape of Y Test :(822,)
binary
binary


In [19]:
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='linear',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))

for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='poly',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    print("Poly SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))
    
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='rbf',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    print("RBF SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))

Linear SVM value of C:1, training score :0.680353 , Test Score: 0.680049 

Linear SVM value of C:3, training score :0.714720 , Test Score: 0.722628 

Linear SVM value of C:5, training score :0.723236 , Test Score: 0.738443 

Linear SVM value of C:10, training score :0.745742 , Test Score: 0.753041 

Linear SVM value of C:40, training score :0.768248 , Test Score: 0.762774 

Linear SVM value of C:60, training score :0.773114 , Test Score: 0.772506 

Linear SVM value of C:80, training score :0.774027 , Test Score: 0.774939 

Linear SVM value of C:100, training score :0.772506 , Test Score: 0.779805 

Poly SVM value of C:1, training score :0.744526 , Test Score: 0.734793 

Poly SVM value of C:3, training score :0.761253 , Test Score: 0.754258 

Poly SVM value of C:5, training score :0.768248 , Test Score: 0.757908 

Poly SVM value of C:10, training score :0.776460 , Test Score: 0.773723 

Poly SVM value of C:40, training score :0.788625 , Test Score: 0.774939 

Poly SVM value of C:60, tra